In [1]:
# This is the tutorial of Force-Matching 

#!/usr/bin/env python3
# Standard python library
import numpy as np
import IO.user_provided
import objective.setup_objective
import optimizer.gradient_free
import objective.compute_objective

# Local library:

# Third party library:

In [2]:
# In the shell, use the slurm job scheduler to request cores to run the job interactively

#[shell]$ srun  -n 2 -t 10:00:00 --x11=first --pty /bin/bash  -l

In [3]:
# Print the content of input file: 
text_file = open("in_obj")
file_content = text_file.read()
print(file_content)
text_file.close()

# units: 

real   

# objective function defined: 

force mW_300K_1bar_500 1 2 2 bf 5000 eng var w 0.0 1.0 

# sampling command  # sampling input file names:  in.force, in.rdf, in.isobar
# sampling by default is LAMMPS 

module load intel && srun -n %d -N1 -c1 --mpi=pmi2 lmp_ml_water < in.%s

# output frequency: current best parameters/obj, restart simplex  

5 5 

#stillinger_weber 6.189  2.3925  1.80  23.15  1.20  -0.333333333333 7.049556277  0.6022245584  4.0  0.0 0.0  
stillinger_weber 6.589  2.0925  1.87  29.15  1.02  -0.303333333333 7.049556277  0.6822245584  3.2  0.4 0.0  

# fit (1) and fix (0) parameters: 

1 1 1 1 1 1 0 1 1 1 0  

# constraints (index lower upper)  ... ): 

10 0 5

# set termination criterion: max number of iteration, tolerance for parameters,tolerance for objective 

16000 1e-16 1e-16

# create (Perturb) or use existing vertices (Restart): 

Nelder-Mead Perturb 

-0.4 -0.4 0.4 0.4 0.4 -0.4 0.4 -0.4 -0.4 



In [4]:
#------------------------------------------------------------------------------
#                       Taking the input from user                             
#------------------------------------------------------------------------------
# global variables:
# main_logger: an object that defines the log file output
# ( you don't have to do anything with it )

# TOTAL_CORES: Number of cores assigned by slurm scheduler
# INPUT: a string of given input file name
# JOBID: a combination of Slurm job id and user-provided id



# For interactive job only:
main_logger,TOTAL_CORES,INPUT,JOBID,Ref,prep= (IO
                                         .user_provided
                                         .from_command_line(
                                            jobID="tutorial",
                                            total_cores=2,
                                            input_file="in_obj")
                                         .finish_reading())



# For command line job only: 
"""
main_logger, TOTAL_CORES, INPUT, JOBID,Ref,prep = (IO
                                          .user_provided
                                          .from_command_line()
                                          .finish_reading())
""" 


'\nmain_logger, TOTAL_CORES, INPUT, JOBID,Ref,prep = (IO\n                                          .user_provided\n                                          .from_command_line()\n                                          .finish_reading())\n'

In [5]:
#------------------------------------------------------------------------------
#                           Set up the workflow                                
#------------------------------------------------------------------------------
# set up working folders
# Initialize sampling methods such LAMMPS
# Parse the arugment of objective functions from the input file

# Output: 
# ref_dict: a python dictionary containing reference data address
# predict_dict, a python dictionary containg predicted data address 
# argument_dict, a python dictionary containing arguments needed to run objective
# functions inclduing number of cores requested, buffersize etc ... 

ref_dict, predict_dict, argument_dict, LAMMPS, last_line = (objective
                                                            .setup_objective
                                                            .setup(
                                                                INPUT,
                                                                TOTAL_CORES,
                                                                JOBID,
                                                                overwrite=True,
                                                                Ref_folder=Ref,
                                                                prep_folder=prep)
                                                            .finish())

In [6]:
#------------------------------------------------------------------------------
#                           Initialize objective functions                     
#------------------------------------------------------------------------------

# Output: 
# instantiate the class object for computing specific objective fucntion  
# eval_objective has a method called "optimize", which takes the input
# of an array of optimized parameters and its type name (a string). 
# This eval_objective will be passed to optimizer 

eval_objective = (objective
                  .compute_objective
                  .prepare(
                    ref_dict,
                    predict_dict,
                    argument_dict,
                    LAMMPS))

In [7]:
#------------------------------------------------------------------------------
#                           start optimization                                 
#------------------------------------------------------------------------------

# Output: 
# optimize_fm: A python object initialized from the input file
# It contains a method called "run_optimization()"

optimize_fm = (optimizer
               .gradient_free
               .NelderMeadSimplex(
                   INPUT,
                   eval_objective,
                   skipped=last_line,
                   Output=JOBID+"/Output"))

# run optimization ...
optimize_fm.run_optimization()



Process ForkPoolWorker-111:
Process ForkPoolWorker-112:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/jguo20/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/project/palmer/Jingxiang/ours_optimization/IO/reader.py", line 159, in read_LAMMPS_traj
    return np.f

KeyboardInterrupt: 